In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter
from time import time
import datetime

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.utils import to_categorical, plot_model
from keras import backend as K
from resnet_0919 import ResNet
from simple_models_0911 import light_cnn, deep_cnn2

%matplotlib inline

Using TensorFlow backend.


In [2]:
"""
Reading all the inputs 
1. Log Spectrogram
2. Mel Spectrogram 
3. MFCCs 
"""
x_train_logspec = np.load('../../../train/x_train.npy')
x_train_melspec = np.load('../../../train/x_train_melspec.npy')
x_train_mfcc = np.load('../../../train/x_train_mfcc.npy')
x_train_mfcc_delta = np.load('../../../train/x_train_mfcc_delta.npy')
x_train_mfcc_delta2 = np.load('../../../train/x_train_mfcc_delta2.npy')

y_train = np.load('../../../train/y_train.npy')

In [3]:
classes = ['yes', 'no', 
           'up', 'down', 
           'left', 'right', 
           'on', 'off', 
           'stop', 'go', 
           'silence', 'unknown']

all_classes = [x for x in classes[:11]]
for ind, cl in enumerate(os.listdir('../../../train/audio/')):
    if cl not in classes:
        all_classes.append(cl)
print(all_classes)

['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'silence', 'bed', 'one', 'sheila', 'two', 'wow', 'marvin', 'four', 'house', 'nine', 'bird', 'eight', 'three', 'dog', '_background_noise_', 'seven', 'cat', 'happy', 'six', 'five', 'tree', 'zero']


In [4]:
"""
There is quite a bit of class imbalance in our data so we are going to calculate class_weights and use it while model fitting
"""
def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: float(majority/count) for cls, count in counter.items()}  

class_weights = get_class_weights(y_train)

In [5]:
class_weights

{11: 1.0,
 7: 17.337980566117448,
 6: 17.337980566117448,
 8: 17.24327731092437,
 4: 17.27957894736842,
 5: 17.41154009333899,
 0: 6.839833333333333,
 9: 17.27957894736842,
 10: 17.26503996634413,
 2: 17.301433389544687,
 3: 17.44113897152571,
 1: 17.396778295888087}

In [6]:
"""
Checking if our inputs are in the correct shape. 
"""
x_train_logspec.shape

(70721, 99, 161, 1)

In [7]:
x_train_melspec=x_train_melspec.reshape(tuple(list(x_train_melspec.shape)+[1]))

In [8]:
x_train_melspec.shape # Seems ok

(70721, 128, 32, 1)

In [9]:
x_train_mfcc.shape

(70721, 13, 32)

In [10]:
"""
We will stack the mfccs and the deltas into one input 
"""
x_train_mfcc_new = np.stack((x_train_mfcc, x_train_mfcc_delta,x_train_mfcc_delta2), axis = -1) 

In [11]:
x_train_mfcc_new.shape # seems ok

(70721, 13, 32, 3)

In [19]:
def build_model(input_type,filter_item=[8,16,32],nn_arch):
    y_train = np.load('../../../train/y_train.npy')
    batch_size = 64
    output_size = 12
    d= datetime.datetime.today()
    if input_type=='log':
        input_spec= x_train_logspec
    elif input_type=='mel':
        input_spec = x_train_melspec
    else: 
        input_spec= x_train_mfcc_new
    date = str(d.month)+'_'+str(d.day)
    arch = input_type+str(nn_arch)+'_'+str(filter_item)
    input_size = input_spec.shape[1:]
    if nn_arch=='Resnet':
        sr = ResNet(filter_item, input_size, output_size)
    if nn_arch == 'light_cnn':
        sr = light_cnn(input_size, output_size)
    if nn_arch == 'deep_cnn':
        sr = deep_cnn2(input_size, output_size)
    sr.build()
    sr.m.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
        # to save a png of the model you need pydot and graphviz installed
    plot_model(sr.m, to_file = '{}_{}.png'.format(arch,date), show_shapes = True)
        #callbacks, remember to make folders to store files 
    checkpointer = ModelCheckpoint(filepath='{}_{}_best.h5'.format(arch, date),verbose=0,save_best_only=True)
   
    earlystopping = EarlyStopping()

    tensorboard = TensorBoard(log_dir = '{}_{}'.format(date, time()), histogram_freq = 0, write_graph = True, write_images = True)
    x_train, x_val, y_train, y_val = train_test_split(input_spec, y_train, test_size=0.1, random_state=2017)
    history = sr.m.fit(x_train, 
                   to_categorical(y_train), 
                   batch_size = batch_size, 
                   epochs = 20, 
                   verbose = 1, shuffle = True, 
                   class_weight = class_weights,
                   validation_data = (x_val, to_categorical(y_val)), 
                   callbacks = [checkpointer, tensorboard]) # add more callbacks if you want
    sr.m.save(arch+'.model')
    

## 1. Resnet Model with Log Spec Data and [8,16,32] sequence of filters

In [20]:
""" Building Resnet Model with Log Spec Data and [8,16,32] sequence of filters """
build_model('log',[8,16,32],'Resnet')

W1025 22:17:42.552090 139667592652544 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1025 22:17:42.581441 139667592652544 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1025 22:17:42.583403 139667592652544 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1025 22:17:42.602551 139667592652544 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The 

Train on 63648 samples, validate on 7073 samples


W1025 22:17:57.612882 139667592652544 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1025 22:17:57.613982 139667592652544 deprecation_wrapper.py:119] From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/20
63648/63648 [==============================] - 105s 2ms/step - loss: 6.3438 - acc: 0.5508 - val_loss: 0.7830 - val_acc: 0.7944
Epoch 2/20
63648/63648 [==============================] - 99s 2ms/step - loss: 2.1867 - acc: 0.8215 - val_loss: 1.1077 - val_acc: 0.7001
Epoch 3/20
63648/63648 [==============================] - 99s 2ms/step - loss: 1.5558 - acc: 0.8771 - val_loss: 0.2847 - val_acc: 0.9088
Epoch 4/20
63648/63648 [==============================] - 99s 2ms/step - loss: 1.2523 - acc: 0.9035 - val_loss: 0.2893 - val_acc: 0.9075
Epoch 5/20
63648/63648 [==============================] - 99s 2ms/step - loss: 1.0815 - acc: 0.9158 - val_loss: 0.2011 - val_acc: 0.9314
Epoch 6/20
63648/63648 [==============================] - 99s 2ms/step - loss: 0.9523 - acc: 0.9274 - val_loss: 0.3386 - val_acc: 0.8983
Epoch 7/20
63648/63648 [==============================] - 99s 2ms/step - loss: 0.8411 - acc: 0.9344 - val_loss: 0.3808 - val_acc: 0.8797
Epoch 8/20
63648/63648 [================

## 2. Resnet Model with Log Spec Data and [16,32,64] sequence of filters

In [21]:
""" Building Resnet Model with Log Spec Data and [8,16,32] sequence of filters """
build_model('log',[16,32,64],'Resnet')

Train on 63648 samples, validate on 7073 samples
Epoch 1/20
63648/63648 [==============================] - 175s 3ms/step - loss: 6.4113 - acc: 0.5699 - val_loss: 4.6274 - val_acc: 0.2203
Epoch 2/20
63648/63648 [==============================] - 172s 3ms/step - loss: 1.9431 - acc: 0.8443 - val_loss: 0.4144 - val_acc: 0.8917
Epoch 3/20
63648/63648 [==============================] - 172s 3ms/step - loss: 1.4343 - acc: 0.8866 - val_loss: 0.7828 - val_acc: 0.7326
Epoch 4/20
63648/63648 [==============================] - 172s 3ms/step - loss: 1.1622 - acc: 0.9068 - val_loss: 0.4890 - val_acc: 0.8582
Epoch 5/20
63648/63648 [==============================] - 172s 3ms/step - loss: 1.0052 - acc: 0.9252 - val_loss: 0.5501 - val_acc: 0.8302
Epoch 6/20
63648/63648 [==============================] - 172s 3ms/step - loss: 0.8541 - acc: 0.9359 - val_loss: 0.3442 - val_acc: 0.8767
Epoch 7/20
63648/63648 [==============================] - 172s 3ms/step - loss: 0.7528 - acc: 0.9429 - val_loss: 0.1236 - v

## 3. Resnet Model with Mel Spec Data and [8,16,32] sequence of filters

In [22]:
""" Building Resnet Model with Mel Spec Data and [8,16,32] sequence of filters """
build_model('mel',[8,16,32],'Resnet')

Train on 63648 samples, validate on 7073 samples
Epoch 1/20
63648/63648 [==============================] - 45s 709us/step - loss: 10.1855 - acc: 0.3733 - val_loss: 1.2113 - val_acc: 0.6157
Epoch 2/20
63648/63648 [==============================] - 42s 657us/step - loss: 3.8613 - acc: 0.6805 - val_loss: 0.8038 - val_acc: 0.7828
Epoch 3/20
63648/63648 [==============================] - 41s 652us/step - loss: 2.7189 - acc: 0.7705 - val_loss: 0.5941 - val_acc: 0.8089
Epoch 4/20
63648/63648 [==============================] - 41s 652us/step - loss: 2.1895 - acc: 0.8179 - val_loss: 0.7597 - val_acc: 0.7359
Epoch 5/20
63648/63648 [==============================] - 41s 652us/step - loss: 1.8344 - acc: 0.8476 - val_loss: 0.3512 - val_acc: 0.8842
Epoch 6/20
63648/63648 [==============================] - 42s 655us/step - loss: 1.6021 - acc: 0.8658 - val_loss: 0.7684 - val_acc: 0.7704
Epoch 7/20
63648/63648 [==============================] - 41s 652us/step - loss: 1.3883 - acc: 0.8842 - val_loss: 0.

## 4. Resnet Model with Mel Spec Data and [16,32,64] sequence of filters

In [23]:
""" Building Resnet Model with Mel Spec Data and [16,32,64] sequence of filters """
build_model('mel',[16,32,64],'Resnet')

Train on 63648 samples, validate on 7073 samples
Epoch 1/20
63648/63648 [==============================] - 72s 1ms/step - loss: 9.1192 - acc: 0.4213 - val_loss: 1.4146 - val_acc: 0.5528
Epoch 2/20
63648/63648 [==============================] - 68s 1ms/step - loss: 3.1739 - acc: 0.7381 - val_loss: 1.2156 - val_acc: 0.6037
Epoch 3/20
63648/63648 [==============================] - 67s 1ms/step - loss: 2.1842 - acc: 0.8252 - val_loss: 0.5797 - val_acc: 0.8159
Epoch 4/20
63648/63648 [==============================] - 67s 1ms/step - loss: 1.7023 - acc: 0.8622 - val_loss: 1.7804 - val_acc: 0.5746
Epoch 5/20
63648/63648 [==============================] - 68s 1ms/step - loss: 1.3766 - acc: 0.8853 - val_loss: 0.6488 - val_acc: 0.7850
Epoch 6/20
63648/63648 [==============================] - 68s 1ms/step - loss: 1.2058 - acc: 0.9022 - val_loss: 0.3868 - val_acc: 0.8735
Epoch 7/20
63648/63648 [==============================] - 68s 1ms/step - loss: 1.0395 - acc: 0.9144 - val_loss: 0.2217 - val_acc:

## 5. Resnet Model with MFCC Data and [8,16,32] sequence of filters

In [24]:
""" Building Resnet Model with MFCC Data and [8,16,32] sequence of filters """
build_model('mfcc',[8,16,32],'Resnet')

Train on 63648 samples, validate on 7073 samples
Epoch 1/20
63648/63648 [==============================] - 40s 629us/step - loss: 7.1522 - acc: 0.5699 - val_loss: 0.8896 - val_acc: 0.6915
Epoch 2/20
63648/63648 [==============================] - 36s 562us/step - loss: 3.2869 - acc: 0.7451 - val_loss: 0.8272 - val_acc: 0.7209
Epoch 3/20
63648/63648 [==============================] - 36s 564us/step - loss: 2.5787 - acc: 0.8001 - val_loss: 0.8656 - val_acc: 0.7171
Epoch 4/20
63648/63648 [==============================] - 36s 563us/step - loss: 2.1634 - acc: 0.8289 - val_loss: 0.5477 - val_acc: 0.8180
Epoch 5/20
63648/63648 [==============================] - 36s 568us/step - loss: 1.8790 - acc: 0.8455 - val_loss: 0.3194 - val_acc: 0.8950
Epoch 6/20
63648/63648 [==============================] - 36s 568us/step - loss: 1.6663 - acc: 0.8660 - val_loss: 0.3935 - val_acc: 0.8750
Epoch 7/20
63648/63648 [==============================] - 36s 568us/step - loss: 1.5032 - acc: 0.8743 - val_loss: 0.5

## 6. Resnet Model with MFCC Data and [16,32,64] sequence of filters

In [25]:
""" Building Resnet Model with MFCC Data and [16,32,64] sequence of filters """
build_model('mfcc',[16,32,64],'Resnet')

Train on 63648 samples, validate on 7073 samples
Epoch 1/20
63648/63648 [==============================] - 39s 612us/step - loss: 5.8573 - acc: 0.6424 - val_loss: 1.0382 - val_acc: 0.6765
Epoch 2/20
63648/63648 [==============================] - 36s 572us/step - loss: 2.5127 - acc: 0.8132 - val_loss: 0.5771 - val_acc: 0.8135
Epoch 3/20
63648/63648 [==============================] - 36s 559us/step - loss: 1.8061 - acc: 0.8637 - val_loss: 0.3813 - val_acc: 0.8817
Epoch 4/20
63648/63648 [==============================] - 36s 561us/step - loss: 1.4028 - acc: 0.8896 - val_loss: 0.3841 - val_acc: 0.8730
Epoch 5/20
63648/63648 [==============================] - 35s 545us/step - loss: 1.1769 - acc: 0.9043 - val_loss: 0.2516 - val_acc: 0.9207
Epoch 6/20
63648/63648 [==============================] - 35s 543us/step - loss: 0.9666 - acc: 0.9180 - val_loss: 0.2778 - val_acc: 0.9214
Epoch 7/20
63648/63648 [==============================] - 35s 553us/step - loss: 0.8221 - acc: 0.9277 - val_loss: 0.4

<br/><br/><br/><br/><br/><br/>

*******************************************

# <span style="color:blue"> Now we implement another approach where we only train on known labels and see if we can do some thresholding on the probability to deal with unknowns </span>

*******************************************

<br/><br/><br/><br/><br/><br/>


In [55]:
def build_model_withoutunk(input_type,nn_arch,filter_item=[8,16,32]):
    y_train = np.load('../../../train/y_train.npy')
    batch_size = 64
    output_size = 11
    d= datetime.datetime.today()
    if input_type=='log':
        input_spec= x_train_logspec
    elif input_type=='mel':
        input_spec = x_train_melspec
    else: 
        input_spec= x_train_mfcc_new
    date = str(d.month)+'_'+str(d.day)
    arch = input_type+'unk'+str(nn_arch)+'_'+str(filter_item)
    
    indexRemove = np.where(y_train==11)
    input_spec = np.delete(input_spec, indexRemove, axis=0)
    y_train = np.delete(y_train, indexRemove)
    
    input_size = input_spec.shape[1:]
    if nn_arch=='Resnet':
        sr = ResNet(filter_item, input_size, output_size)
    if nn_arch == 'light_cnn':
        sr = light_cnn(input_size, output_size)
    if nn_arch == 'deep_cnn':
        sr = deep_cnn2(input_size, output_size)
    sr.build()
    sr.m.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
        # to save a png of the model you need pydot and graphviz installed
    plot_model(sr.m, to_file = '{}_{}.png'.format(arch,date), show_shapes = True)
        #callbacks, remember to make folders to store files 
    checkpointer = ModelCheckpoint(filepath='{}_{}_best.h5'.format(arch, date),verbose=0,save_best_only=True)
   
    earlystopping = EarlyStopping()
    
    tensorboard = TensorBoard(log_dir = '{}_{}'.format(date, time()), histogram_freq = 0, write_graph = True, write_images = True)
    # Removing the unknown label data so that our network trains exclusively on known labels #
    
    x_train, x_val, y_train, y_val = train_test_split(input_spec, y_train, test_size=0.1, random_state=2017)
    history = sr.m.fit(x_train, 
                   to_categorical(y_train), 
                   batch_size = batch_size, 
                   epochs = 20, 
                   verbose = 1, shuffle = True, 
                   #class_weight = class_weights,
                   validation_data = (x_val, to_categorical(y_val)), 
                   callbacks = [checkpointer, tensorboard]) # add more callbacks if you want
    sr.m.save(arch+'.model')

# 1a.  Resnet Model with Log Spec Data and [8,16,32] sequence of filters with knowns only 

In [56]:
""" Building Resnet Model with Log Spec Data and [8,16,32] sequence of filters with KNOWNS only """
build_model_withoutunk('log','Resnet',[8,16,32])

Train on 26713 samples, validate on 2969 samples
Epoch 1/20
26713/26713 [==============================] - 46s 2ms/step - loss: 1.1354 - acc: 0.5981 - val_loss: 1.9047 - val_acc: 0.6063
Epoch 2/20
26713/26713 [==============================] - 42s 2ms/step - loss: 0.3077 - acc: 0.8995 - val_loss: 3.0180 - val_acc: 0.6319
Epoch 3/20
26713/26713 [==============================] - 42s 2ms/step - loss: 0.1964 - acc: 0.9370 - val_loss: 0.5732 - val_acc: 0.8377
Epoch 4/20
26713/26713 [==============================] - 42s 2ms/step - loss: 0.1526 - acc: 0.9504 - val_loss: 0.5724 - val_acc: 0.8360
Epoch 5/20
26713/26713 [==============================] - 42s 2ms/step - loss: 0.1319 - acc: 0.9579 - val_loss: 0.2102 - val_acc: 0.9333
Epoch 6/20
26713/26713 [==============================] - 42s 2ms/step - loss: 0.1175 - acc: 0.9607 - val_loss: 0.4214 - val_acc: 0.8794
Epoch 7/20
26713/26713 [==============================] - 42s 2ms/step - loss: 0.1030 - acc: 0.9651 - val_loss: 0.3439 - val_acc:

# 2a.  Resnet Model with Log Spec Data and [16,32,64] sequence of filters with knowns only 

In [57]:
""" Building Resnet Model with Log Spec Data and [16,32,64] sequence of filters with KNOWNS only """
build_model_withoutunk('log','Resnet',[16,32,64])

Train on 26713 samples, validate on 2969 samples
Epoch 1/20
26713/26713 [==============================] - 77s 3ms/step - loss: 1.0110 - acc: 0.6499 - val_loss: 8.1367 - val_acc: 0.2560
Epoch 2/20
26713/26713 [==============================] - 73s 3ms/step - loss: 0.2566 - acc: 0.9163 - val_loss: 0.8908 - val_acc: 0.8164
Epoch 3/20
26713/26713 [==============================] - 73s 3ms/step - loss: 0.1666 - acc: 0.9460 - val_loss: 2.0874 - val_acc: 0.7363
Epoch 4/20
26713/26713 [==============================] - 73s 3ms/step - loss: 0.1373 - acc: 0.9556 - val_loss: 1.1244 - val_acc: 0.7359
Epoch 5/20
26713/26713 [==============================] - 73s 3ms/step - loss: 0.1161 - acc: 0.9625 - val_loss: 0.6157 - val_acc: 0.8528
Epoch 6/20
26713/26713 [==============================] - 73s 3ms/step - loss: 0.1042 - acc: 0.9655 - val_loss: 0.3944 - val_acc: 0.8804
Epoch 7/20
26713/26713 [==============================] - 73s 3ms/step - loss: 0.0858 - acc: 0.9727 - val_loss: 2.0139 - val_acc:

# 3a.  Resnet Model with Mel Spec Data and [8,16,32] sequence of filters with knowns only 

In [59]:
""" Building Resnet Model with Mel Spec Data and [8,16,32] sequence of filters with KNOWNS only """
build_model_withoutunk('mel','Resnet',[8,16,32])

Train on 26713 samples, validate on 2969 samples
Epoch 1/20
26713/26713 [==============================] - 23s 860us/step - loss: 1.7219 - acc: 0.4548 - val_loss: 2.4178 - val_acc: 0.4055
Epoch 2/20
26713/26713 [==============================] - 18s 688us/step - loss: 0.7945 - acc: 0.7237 - val_loss: 2.8695 - val_acc: 0.4453
Epoch 3/20
26713/26713 [==============================] - 18s 683us/step - loss: 0.4418 - acc: 0.8512 - val_loss: 0.9719 - val_acc: 0.7181
Epoch 4/20
26713/26713 [==============================] - 18s 685us/step - loss: 0.3194 - acc: 0.8938 - val_loss: 0.4298 - val_acc: 0.8572
Epoch 5/20
26713/26713 [==============================] - 18s 689us/step - loss: 0.2581 - acc: 0.9154 - val_loss: 0.5998 - val_acc: 0.8296
Epoch 6/20
26713/26713 [==============================] - 18s 684us/step - loss: 0.2174 - acc: 0.9273 - val_loss: 0.3183 - val_acc: 0.8949
Epoch 7/20
26713/26713 [==============================] - 18s 687us/step - loss: 0.1894 - acc: 0.9368 - val_loss: 0.4

# 4a Resnet Model with Mel Spec Data and [16,32,64] sequence of filters with knowns only

In [60]:
""" Building Resnet Model with Mel Spec Data and [16,32,64] sequence of filters with KNOWNS only """
build_model_withoutunk('mel','Resnet',[16,32,64])

Train on 26713 samples, validate on 2969 samples
Epoch 1/20
26713/26713 [==============================] - 34s 1ms/step - loss: 1.1791 - acc: 0.6053 - val_loss: 8.2847 - val_acc: 0.3594
Epoch 2/20
26713/26713 [==============================] - 29s 1ms/step - loss: 0.3590 - acc: 0.8795 - val_loss: 0.5096 - val_acc: 0.8501
Epoch 3/20
26713/26713 [==============================] - 29s 1ms/step - loss: 0.2474 - acc: 0.9186 - val_loss: 0.6909 - val_acc: 0.8067
Epoch 4/20
26713/26713 [==============================] - 29s 1ms/step - loss: 0.1869 - acc: 0.9386 - val_loss: 0.3905 - val_acc: 0.8909
Epoch 5/20
26713/26713 [==============================] - 29s 1ms/step - loss: 0.1558 - acc: 0.9498 - val_loss: 0.7290 - val_acc: 0.8141
Epoch 6/20
26713/26713 [==============================] - 29s 1ms/step - loss: 0.1327 - acc: 0.9557 - val_loss: 0.3759 - val_acc: 0.8942
Epoch 7/20
26713/26713 [==============================] - 29s 1ms/step - loss: 0.1145 - acc: 0.9620 - val_loss: 0.2117 - val_acc:

# 5a Resnet Model with MFCC Data and [8,16,32] sequence of filters with knowns only

In [61]:
""" Building Resnet Model with Mel Spec Data and [8,16,32] sequence of filters with KNOWNS only """
build_model_withoutunk('mfcc','Resnet',[8,16,32])

Train on 26713 samples, validate on 2969 samples
Epoch 1/20
26713/26713 [==============================] - 21s 795us/step - loss: 0.9929 - acc: 0.6709 - val_loss: 0.9241 - val_acc: 0.7292
Epoch 2/20
26713/26713 [==============================] - 16s 615us/step - loss: 0.4157 - acc: 0.8597 - val_loss: 0.4481 - val_acc: 0.8518
Epoch 3/20
26713/26713 [==============================] - 16s 606us/step - loss: 0.3048 - acc: 0.8990 - val_loss: 0.4281 - val_acc: 0.8676
Epoch 4/20
26713/26713 [==============================] - 16s 606us/step - loss: 0.2502 - acc: 0.9187 - val_loss: 0.3338 - val_acc: 0.8946
Epoch 5/20
26713/26713 [==============================] - 16s 605us/step - loss: 0.2097 - acc: 0.9301 - val_loss: 0.3320 - val_acc: 0.8915
Epoch 6/20
26713/26713 [==============================] - 17s 624us/step - loss: 0.1756 - acc: 0.9403 - val_loss: 0.3105 - val_acc: 0.9128
Epoch 7/20
26713/26713 [==============================] - 16s 608us/step - loss: 0.1582 - acc: 0.9486 - val_loss: 0.4

# 1f Resnet Model with MFCC Data and [16,32,64] sequence of filters with knowns only

In [62]:
""" Building Resnet Model with Mel Spec Data and [16,32,64] sequence of filters with KNOWNS only """
build_model_withoutunk('mfcc','Resnet',[16,32,64])

Train on 26713 samples, validate on 2969 samples
Epoch 1/20
26713/26713 [==============================] - 21s 788us/step - loss: 0.7887 - acc: 0.7700 - val_loss: 0.9881 - val_acc: 0.7457
Epoch 2/20
26713/26713 [==============================] - 16s 592us/step - loss: 0.2990 - acc: 0.8987 - val_loss: 0.4257 - val_acc: 0.8740
Epoch 3/20
26713/26713 [==============================] - 16s 583us/step - loss: 0.2154 - acc: 0.9275 - val_loss: 0.4404 - val_acc: 0.8653
Epoch 4/20
26713/26713 [==============================] - 15s 579us/step - loss: 0.1724 - acc: 0.9418 - val_loss: 0.7018 - val_acc: 0.8346
Epoch 5/20
26713/26713 [==============================] - 16s 581us/step - loss: 0.1371 - acc: 0.9544 - val_loss: 0.5058 - val_acc: 0.8670
Epoch 6/20
26713/26713 [==============================] - 15s 580us/step - loss: 0.1138 - acc: 0.9604 - val_loss: 0.3037 - val_acc: 0.9178
Epoch 7/20
26713/26713 [==============================] - 16s 602us/step - loss: 0.0938 - acc: 0.9687 - val_loss: 0.3